# Optimize and Deploy AI Models with Pruna and Hugging Face

Objective: Build a complete tutorial demonstrating how to optimize the [Efficient-Large-Model/Sana_600M_1024px_ControlNet_HED](https://huggingface.co/Efficient-Large-Model/Sana_600M_1024px_ControlNet_HED) diffusion model using Pruna and deploy it seamlessly to the Hugging Face Hub.

Model: [Efficient-Large-Model/Sana_600M_1024px_ControlNet_HED](https://huggingface.co/Efficient-Large-Model/Sana_600M_1024px_ControlNet_HED)

Dataset: [data-is-better-together/open-image-preferences-v1-binarized](https://huggingface.co/datasets/data-is-better-together/open-image-preferences-v1-binarized)

To follow along, ensure that you have the Pruna SDK installed along with all required third-party libraries. Running this tutorial in a clean virtual environment is recommended for a smooth setup.

<a target="_blank" href="https://colab.research.google.com/github/PrunaAI/pruna/blob/v|version|/docs/tutorials/deploying_sana_tutorial.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
pip install pruna 

In [ ]:
pip install datasets huggingface_hub gradio 

You will need to login on the Hugging Face Hub for using the model weights. We also need to select the best available device for executing the notebook. Run the cells below to do the same.

In [1]:
from huggingface_hub import notebook_login

notebook_login()

Token has not been saved to git credential helper.


In [3]:
import torch

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

## 1. Smash Configuration

To optimize the model effectively, we first define the configuration methods that enhance its performance. For detailed options and parameter explanations, refer to the [SmashConfig guide](https://docs.pruna.ai/en/stable/docs_pruna/user_manual/configure.html).

In this tutorial, we will:

* Select **hqq_diffusers** to reduce memory usage during inference.
* Set the **weight bit precision** to 8 bits for the diffusers model:
* Upload the optimized (“smashed”) model to the Hugging Face Hub for easy access, sharing, and deployment in downstream applications.

In [ ]:
import torch
from diffusers import SanaPipeline

from pruna import PrunaModel, SmashConfig, smash

# Define the model ID
model_id = "Efficient-Large-Model/Sana_600M_512px_diffusers"

# Load the pre-trained model
pipe = SanaPipeline.from_pretrained(model_id, variant="fp16", torch_dtype=torch.float16)
pipe = pipe.to(device)

# 2. Configure Pruna smash
smash_config = SmashConfig()
smash_config["quantizer"] = "hqq_diffusers"  # Quantizer to reduce memory usage
smash_config["hqq_diffusers_weight_bits"] = 8

# 3. Smash (optimize) the model
smashed_pipe = smash(model=pipe, smash_config=smash_config)

# 4. Push the smashed pipeline to Hugging Face Hub using save_to_hub
smashed_pipe.save_to_hub("AINovice2005/Sana_600M_ControlNet_HED-smashed")

print("✅ Smashed Sana model uploaded successfully to Hugging Face Hub.")

## **2. Load and Collate Dataset**

In this step, we will load the dataset required for optimizing and evaluating the model. This dataset will provide the input data needed to assess the model’s performance after applying optimization techniques such as quantization.

We will use the [`data-is-better-together/open-image-preferences-v1-binarized`](https://huggingface.co/datasets/data-is-better-together/open-image-preferences-v1-binarized) dataset, which contains binarized user image preferences and prompts for image generation tasks. Correctly loading and collating the dataset ensures that the input is properly prepared, enabling smooth evaluation.

In [1]:
from pruna import SmashConfig
from pruna.data.utils import split_train_into_train_val_test
from datasets import load_dataset, Image
import numpy as np
import torch

# Load image dataset
dataset = load_dataset("data-is-better-together/open-image-preferences-v1-binarized")["train"]

dataset = dataset.rename_column("chosen", "image")
dataset = dataset.rename_column("prompt", "text")

# Cast 'image' column to Image type so outputs are PIL Images
dataset = dataset.cast_column("image", Image())

# Select only first 500 images
dataset = dataset.select(range(500))

train_ds, val_ds, test_ds = split_train_into_train_val_test(dataset, seed=42)

# Add to SmashConfig
smash_config = SmashConfig()
smash_config.add_data(
    (train_ds, val_ds, test_ds),
    collate_fn="image_generation_collate"  # Replace with your image collate function name
)

/workspaces/pruna/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Multiple distributions found for package optimum. Picked distribution: optimum
INFO - Loaded only training, splitting train 80/10/10 into train, validation and test...
INFO - Using best available device: 'cpu'


ValueError: The following required parameters are missing in collate_fn_args: ['img_size']

## 3. Evaluate the Model

Now that the model and dataset are set up, we can proceed to evaluate the model using the **Pruna Evaluation Agent**. This evaluation helps us measure the model’s current performance before optimization, providing a baseline for comparison. It assesses how well the model performs on the given dataset and generates relevant metrics that will guide us in understanding the impact of our optimization configurations later.

In [9]:
# Import required modules from Pruna
from pruna.data.pruna_datamodule import PrunaDataModule
from pruna.evaluation.evaluation_agent import EvaluationAgent
from pruna.evaluation.metrics import (
    LatencyMetric,
    TotalTimeMetric,
)
from pruna.evaluation.task import Task
from pruna import PrunaModel

# Load the smashed (optimized) model pipeline from Hugging Face Hub
smashed_pipe = PrunaModel.from_hub("AINovice2005/Sana_600M_ControlNet_HED-smashed")

# Define evaluation metrics (example: total time and latency)
metrics = [
    TotalTimeMetric(n_iterations=1, n_warmup_iterations=1),
    LatencyMetric(n_iterations=1, n_warmup_iterations=1),
]

# Define the evaluation task with metrics and datamodule
# (Ensure `datamodule` and `device` are defined before this script runs)
task = Task(metrics, datamodule=datamodule, device=device)

# Initialize the evaluation agent
eval_agent = EvaluationAgent(task)

# Move smashed model to evaluation device (GPU or CPU)
smashed_pipe.move_to_device(device)

# Evaluate the smashed model pipeline using the evaluation agent
smashed_model_results = eval_agent.evaluate(smashed_pipe)

# Optionally, print results for verification
print(smashed_model_results)


Fetching 18 files: 100%|██████████| 18/18 [00:00<00:00, 61082.10it/s]
INFO - Using best available device: 'cuda'
WARNING - Argument cache_dir not found in config file. Skipping...


Loading pipeline components...: 100%|██████████| 4/4 [00:04<00:00,  1.20s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 82.00 MiB. GPU 0 has a total capacity of 21.96 GiB of which 63.12 MiB is free. Process 20197 has 21.89 GiB memory in use. Of the allocated memory 21.60 GiB is allocated by PyTorch, and 76.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [8]:
import gc
import torch

# Run garbage collector to free Python memory references
gc.collect()

# Empty cached GPU memory that is no longer needed
torch.cuda.empty_cache()

# Optionally, if using PyTorch >=1.9, clear CUDA’s allocator memory as well
torch.cuda.ipc_collect()


## 4. Gradio Demo

Once the model has been optimized, we can deploy the smashed model using **Gradio** to create an interactive demo. This allows anyone to test the model’s capabilities directly in their browser.

In this section, we will:

* Show how to deploy the optimized model on the Hugging Face Hub with a Gradio demo
* Discuss considerations such as **handling queuing**, especially if multiple users access the demo simultaneously
* Highlight best practices for integrating Gradio demos in your Hugging Face Space to ensure a smooth and responsive user experience

Creating a Gradio demo not only showcases your optimized model effectively but also enables easy sharing and real-world testing by the community.

In [1]:
import gradio as gr

from pruna import PrunaModel

# Load PrunaModel
model = PrunaModel.from_hub("AINovice2005/Sana_600M_ControlNet_HED-smashed")


# Inference function
def generate_image(prompt):
    """Generate an image from a given text prompt."""
    result = pipe(prompt, num_inference_steps=25, guidance_scale=7.5)
    return result.images[0]


# Create Gradio interface with queueing enabled
demo = gr.Interface(
    fn=generate_image,
    inputs=gr.Textbox(lines=2, placeholder="Enter your prompt here...", label="Prompt"),
    outputs=gr.Image(type="pil"),
    title="Sana Smashed Text-to-Image Demo",
    description="Generate high-quality images using the smashed Sana diffusion model optimized with Pruna.",
    allow_flagging="never",
)

# Enable queueing to handle multiple users
demo.queue()

# Launch the app
if __name__ == "__main__":
    demo.launch()

Multiple distributions found for package optimum. Picked distribution: optimum


Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

INFO - Using best available device: 'cuda'
WARNING - Argument cache_dir not found in config file. Skipping...
/teamspace/studios/this_studio/.venv/lib/python3.10/site-packages/diffusers/configuration_utils.py:248: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a model, please use <class 'diffusers.models.transformers.sana_transformer.SanaTransformer2DModel'>.load_config(...) followed by <class 'diffusers.models.transformers.sana_transformer.SanaTransformer2DModel'>.from_config(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)
100%|██████████| 230/230 [00:00<00:00, 7521.71it/s]


Loading pipeline components...:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

WARNING - Could not move model to device: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.
/teamspace/studios/this_studio/.venv/lib/python3.10/site-packages/gradio/interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Conclusions
In this tutorial, we have covered the end-to-end workflow for optimizing and evaluating a text-to-image diffusion model using Pruna.

We began by loading the Sana base model and defining the SmashConfig with the desired optimization algorithms and parameters. We then smashed the base model, obtaining an optimized version, and ensured its performance improvements by running an evaluation with the EvaluationAgent.

After optimization, we demonstrated how to deploy the smashed model using Gradio to create an interactive demo. This enables anyone to test the model’s capabilities directly in their browser. This end-to-end approach makes it easy to explore trade-offs, iterate on optimization configurations, and deploy robust, production-ready text-to-image models. 

Check out our other [tutorials](https://docs.pruna.ai/en/stable/docs_pruna/tutorials/index.html) for more examples on optimizing and evaluating large language models, text-to-video models using Pruna.